In [28]:
import pandas as pd
import numpy as np

# Data Cleaning
We will now clean the data we scraped from Tagvenue. 

**Important Note!** -> if re-running the script after scraping new data, you must ensure the 'price_type' column contains the combined prices types 'hire fee + min. spend' and 'hire fee + per person' **only**. If there are new combined prices or they no longer have the exact same wording, then you need to check the price cleaning section and make sure it is still compatible. 

Jump to [here](#pricing_strings) to see the unique values of *price_type*

Jump to [here](#pricing_clean) to see the functions that may be incompatible with changes to *price_type*

## Setup

In [29]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 0

## Importing Data
Below we import the general venue data and the prices data as dataframes. Metadata for both tables can be found in the repo Readme. 

In [30]:
venues = pd.read_csv('../data/raw_scraped_data/tag_venue_space_data_31-Jul-22.csv')
prices = pd.read_csv('../data/raw_scraped_data/tag_venue_space_prices_31-Jul-22.csv')

## Cleaning Prices Data
### Overview
We will start by cleaning the prices data which is previewed below: 

In [31]:
prices.head()

,space_url,venue_url,venue_name,space_name,latitude,longitude,day_of_week,pricing_period,time_from,time_to,price,price_type
0,https://www.tagvenue.com/rooms/london/19171/th...,https://www.tagvenue.com/venues/london/2034/th...,the golden hinde,entire ship,51.507014,-0.090456,Monday,Per hour,6:00 –,0:00,£360,hire fee per hour
1,https://www.tagvenue.com/rooms/london/19171/th...,https://www.tagvenue.com/venues/london/2034/th...,the golden hinde,entire ship,51.507014,-0.090456,Tuesday,Per hour,6:00 –,0:00,£360,hire fee per hour
2,https://www.tagvenue.com/rooms/london/19171/th...,https://www.tagvenue.com/venues/london/2034/th...,the golden hinde,entire ship,51.507014,-0.090456,Wednesday,Per hour,6:00 –,0:00,£360,hire fee per hour
3,https://www.tagvenue.com/rooms/london/19171/th...,https://www.tagvenue.com/venues/london/2034/th...,the golden hinde,entire ship,51.507014,-0.090456,Thursday,Per hour,6:00 –,0:00,£360,hire fee per hour
4,https://www.tagvenue.com/rooms/london/19171/th...,https://www.tagvenue.com/venues/london/2034/th...,the golden hinde,entire ship,51.507014,-0.090456,Friday,Per hour,6:00 –,0:00,£360,hire fee per hour


In [32]:
prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54662 entries, 0 to 54661
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   space_url       54662 non-null  object 
 1   venue_url       54662 non-null  object 
 2   venue_name      54662 non-null  object 
 3   space_name      54662 non-null  object 
 4   latitude        54662 non-null  float64
 5   longitude       54662 non-null  float64
 6   day_of_week     54662 non-null  object 
 7   pricing_period  54662 non-null  object 
 8   time_from       51736 non-null  object 
 9   time_to         51736 non-null  object 
 10  price           51736 non-null  object 
 11  price_type      51736 non-null  object 
dtypes: float64(2), object(10)
memory usage: 5.0+ MB


In [33]:
prices.describe(include = 'all')

,space_url,venue_url,venue_name,space_name,latitude,longitude,day_of_week,pricing_period,time_from,time_to,price,price_type
count,54662,54662,54662,54662,54662.000000,54662.000000,54662,54662,51736,51736,51736,51736
unique,4761,1740,1736,3216,NaN,NaN,7,7,38,46,1118,6
top,https://www.tagvenue.com/rooms/london/19621/fi...,https://www.tagvenue.com/venues/london/44/30-e...,30 euston square,whole venue,NaN,NaN,Wednesday,Per day,9:00 –,17:00,£500,hire fee
freq,56,408,408,3133,NaN,NaN,8212,15727,11574,9309,2485,17770
mean,NaN,NaN,NaN,NaN,51.511276,-0.120040,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,0.028975,0.063452,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,51.326833,-0.443729,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,51.503983,-0.144529,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,51.513528,-0.122588,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,51.522738,-0.084337,NaN,NaN,NaN,NaN,NaN,NaN


### Cleaning URLs
Below we check the space_url and venue_url columns contain no spaces and all start with 'https:://www.tagvenue.com'

In [34]:
# Confirm that no rows contain a ' ' character, should return no rows  
prices[prices.space_url.str.contains(" ")]

,space_url,venue_url,venue_name,space_name,latitude,longitude,day_of_week,pricing_period,time_from,time_to,price,price_type


In [35]:
# Confirm that no rows contain a ' ' character, should return no rows  
prices[prices.venue_url.str.contains(" ")]

,space_url,venue_url,venue_name,space_name,latitude,longitude,day_of_week,pricing_period,time_from,time_to,price,price_type


In [36]:
# Confirm all rows start with 'https://www.tagvenue.com'
assert(prices.space_url.str.contains(r'^https://www.tagvenue.com')).all()

In [37]:
# Confirm all rows start with 'https://www.tagvenue.com'
assert(prices.venue_url.str.contains(r'^https://www.tagvenue.com')).all()

### Cleaning Venue and Space Names
Below we remove leading or trailing whitspace from the venue_name and space_name columns: 

In [38]:
prices.venue_name = prices.venue_name.str.strip()
prices.space_name = prices.space_name.str.strip()

### Cleaning Longitude and Latitude
Below we check that the latitude and longitude values are all within the max and min latitude and longitude values defined in our web scrape: 

In [39]:
# Max and min latitude and longiotude values from web scrape:
latitude_min = 51.326626 
latitude_max = 51.7297765
longitude_min = -0.446500003
longitude_max = 0.2190751

assert(prices.latitude.min() > latitude_min)
assert(prices.latitude.max() < latitude_max)
assert(prices.longitude.min() > longitude_min)
assert(prices.longitude.max() < longitude_max)

<a id='pricing_strings'></a>
### Cleaning Pricing Strings
Below we review all string values taken by each of the following columns: 
- day_of_week
- pricing_period
- price_type

In [40]:
prices.day_of_week.unique()

array(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday',
       'Sunday'], dtype=object)

In [41]:
prices.pricing_period.unique()

array(['Per hour', 'Per day', 'closed', 'Per evening', 'Per session',
       'Per morning', 'Per afternoon'], dtype=object)

In [42]:
prices.price_type.unique()

array(['hire fee per hour', 'hire fee', 'per person', nan, 'min. spend',
       'hire fee + min. spend', 'hire fee + per person'], dtype=object)

**Observations:** 

All columns have a small number of tidy strings that don't require cleaning. The nan values in *price_type* were to be expected, that is the value if the space is closed on a particular day (if 'pricing_period' is 'closed'). 

We need to confirm that the nan values occur across the final four columns of the prices dataframe only when the pricing_period is set to 'closed'.   

In [43]:
# filter on rows with pricing_period = 'closed'
closed_rows = prices[(prices.pricing_period == 'closed')]
# Drops 'closed' rows from dataframe
dropped_closed_rows = prices.drop(closed_rows.index)
# Display all rows in dataframe with a nan value
dropped_closed_rows[dropped_closed_rows.isna().any(axis=1)]

,space_url,venue_url,venue_name,space_name,latitude,longitude,day_of_week,pricing_period,time_from,time_to,price,price_type


We have shown that after removing rows with pricing_period set to 'closed' that there are no more nan values in the dataframe. Thus all the nan values correspond with a day the space is closed and thus all the nan values were to be expected. We will remove these rows in our prices dataframe. 

In [44]:
prices = dropped_closed_rows
prices = prices.reset_index(drop = True)

### Cleaning Time Form and To
Firstly, we will clean the time_from column to remove the trailing '-' strings 

In [45]:
# Remove '-' from time_from and any remaining whitespace
prices.time_from = prices.time_from.str.replace('–','').str.strip()

Now we will convert the time data into floats e.g. '5:30' becomes '5.5'. We decided this was the easiest way to manipulate and graph the times.  

In [46]:
# Convert time to datetime object
from_time = pd.to_datetime(prices['time_from'], format='%H:%M')
# Extract and combine hours and minutes data into decimal form. 
prices.time_from = from_time.dt.hour + from_time.dt.minute / 60 

to_time = pd.to_datetime(prices['time_to'], format='%H:%M')
prices.time_to = to_time.dt.hour + to_time.dt.minute / 60 

Finally, we will calculate the time period between the time_from and time_to columns (in hours). Note that in this calculation, we calculate *time_to* - *time_from*. This sometimes gives a negative answer due to midnight going back to 0 in a 24 hour clock e.g. if time_to = 0.0 (midnight) and time_from = 6.0 (6 am) then the subtraction gives us -6. To get the correct answer we add 24 e.g. this becomes 18.  

In [47]:
# subtract time_to and time_from
prices.insert(10, 'time_period', prices.time_to - prices.time_from)
# Where the answer was negative, add 24
prices['time_period'] = np.where(prices.time_period < 0, 
                                 prices.time_period + 24, 
                                 prices.time_period)

<a id='pricing_clean'></a>
### Cleaning Price and Price Type
We will now clean the price and price type columns. There are 2 complicated cases for price and price type. When Price type is equal to 'hire fee + min. spend' or 'hire fee + per person' then the price is recorded as '£price + £price' where each price corresponds to the prices types. An example is shown below: 

**price_type** -> 'hire fee + min. spend'

**price** -> '£300 + £400' 

In this example, the hire fee is £300 and the minimum spend is £400. For all other price_types, there is only a single price provided e.g. 'per person' price_type might have a price of '£50'. 

We wish to create a new column, 'total_price' that has the total price rather than 2 prices being added together. e.g. '£300 + £400' becomes '£700'. But we also want to preserve the individual costs of hire fee and minimum spend / per person. As such, we will also create 3 new columns that contain the individual contributions to total_price of hire fee, minimum spend and per person respectively. These columns will be null for all rows except those using either 'hire fee + min. spend' or 'hire fee + per person' as the price type.

Below we build the new columns and replace the 'price' column with 'total_price'. 

In [49]:
def split_cols(price, type_):
    """Return list of cleaned price data.  
    
    If the price is a combination of prices, it will calculate the total
    price and also return the individual contributions of rent price, per 
    person and min spend."""
    # remove '£' from prices
    price = price.replace('£','')
    # checks if this price contains a '+' 
    if '+' in price:
        # splits prices between '+' into list
        price_list = price.split('+')
        # splits price type between '+' into list
        type_list = type_.split('+')
        total_price = float(price_list[0]) + float(price_list[1])
        # first price is always rent price (format is
        # '£rent price + £per person OR min. spend')
        rent_price = float(price_list[0])
        # If type includes 'min. spend' then the second price is minimum spend 
        if 'min.spend' in type_:
            min_spend = float(price_list[1])
            per_person = np.nan
        # Otherewise the second price must be per person
        else: 
            min_spend = np.nan
            per_person = float(price_list[1])
        return [total_price, type_, rent_price, min_spend, per_person]
    # returns just type and price, leaving rest of list nan if the 
    # price is not combined   
    return [float(price), type_, np.nan, np.nan, np.nan]
    
# Apply split_cols to the price and price_type columns
cleaned_price_data = [split_cols(price, type_)for price, type_ 
                      in zip(prices.price, prices.price_type)]
# Create the new columns we want
cleaned_price_df = pd.DataFrame(cleaned_price_data, 
                                columns = ['total_price', 'price_type', 
                                           'rent_price_split', 
                                           'min_spend_split', 
                                           'per_person_split'])
# drops columns we dont want anymore
prices.drop(columns = ['price', 'price_type'], inplace = True)
# Add new columns to data
prices = prices.join(cleaned_price_df)
prices.head()

AttributeError: 'DataFrame' object has no attribute 'price'

## Cleaning General Venue Data
We will now clean the general venues data previewed below: 

In [50]:
venues.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4761 entries, 0 to 4760
Data columns (total 92 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   space_url                                          4761 non-null   object 
 1   venue_url                                          4761 non-null   object 
 2   venue_name                                         4761 non-null   object 
 3   space_name                                         4761 non-null   object 
 4   latitude                                           4761 non-null   float64
 5   longitude                                          4761 non-null   float64
 6   address                                            4761 non-null   object 
 7   nearest_tube_station                               4425 non-null   object 
 8   max_seated                                         4761 non-null   int64  
 9   max_stan

In [51]:
venues.describe(include = 'all')

,space_url,venue_url,venue_name,space_name,latitude,longitude,address,nearest_tube_station,max_seated,max_standing,area_in_m2,catering_offered,external_catering_allowed,supervenue,Standing_max,Dining_max,Theatre_max,Boardroom_max,Classroom_max,Cabaret_max,U-Shaped_max,Approved caterers only,BYO alcohol allowed,BYO alcohol not allowed,External catering allowed,External catering not allowed,In-house catering,No in-house catering,Venue doesn’t provide alcohol,Venue provides alcohol,Alcohol licence until 23:00,Corkage fee for BYO alcohol,Buyout fee for external catering,Kitchen facilities available for guests,Complimentary water,Complimentary tea and coffee,Alcohol licence until 23:00 (extension available),Halal menu,Kosher menu,Extensive vegan menu,...,Alcohol licence until 22:30 (extension available),Alcohol licence until 3:00,Alcohol licence until 0:30 (extension available),Alcohol licence until 4:00,Alcohol licence until 23:30,Alcohol licence until 23:30 (extension available),Alcohol licence until 21:30,Alcohol licence until 4:00 (extension available),Alcohol licence until 2:30,Alcohol licence until 0:30,Alcohol licence until 22:00 (extension available),Alcohol licence until 5:00,Alcohol licence until 21:00 (extension available),Alcohol licence until 2:30 (extension available),Alcohol licence until 21:00,Alcohol licence until 1:30 (extension available),Wi-Fi,Projector,Flipchart,Natural light,Flatscreen TV,Whiteboard,Conference call facilities,Air conditioning,Storage space,Accommodation available,Parking available,Own music allowed,Bring your own DJ,PA system / music speakers available,Wheelchair accessible,Promoted / ticketed events,Loud music / events,Wedding licence,Temporary event notices (TENs) available,Paid parking facilities available nearby,Disabled access toilets,Ground level,Lift to all floors,Free parking is available on-site
count,4761,4761,4761,4761,4761.000000,4761.000000,4761,4425,4761.000000,4761.000000,4761.000000,4761,4761,4761,3788.000000,3247.000000,1938.000000,2122.000000,1300.000000,1408.000000,1170.000000,4761.000000,4761.000000,4761.000000,4761.000000,4761.000000,4761.000000,4761.000000,4761.000000,4761.000000,529,1155,1156,1156,4761.000000,4761.000000,469,3902,3902,3902,...,16,89,26,22,96,82,10,16,8,36,18,4,9,3,7,2,4761,4761,4761,4761,4761,4761,4761,4761,4761,4761,2667,4761,4761,4761,4761,4761,4761,4761,4761,1186,873,628,919,726
unique,4761,1740,1736,3216,NaN,NaN,1718,899,NaN,NaN,NaN,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,2,2,NaN,NaN,1,2,2,2,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,1,2,2,2,2,2,2,2,2,1,1,1,1,1
top,https://www.tagvenue.com/rooms/london/19171/th...,https://www.tagvenue.com/venues/london/44/30-e...,30 euston square,whole venue,NaN,NaN,"30 Euston Square, Euston, London, NW1 2FB",Farringdon Station (300 yd),NaN,NaN,NaN,Venue offers catering,External catering not allowed,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,False,False,NaN,NaN,True,False,False,True,...,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,False,True,False,False,False,True,False,False,False,True,False,True,True,False,False,False,False,True,True,True,True,True
freq,1,24,24,329,NaN,NaN,24,42,NaN,NaN,NaN,3902,3605,4201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,529,905,900,642,NaN,NaN,469,2008,2710,2077,...,16,89,26,22,96,82,10,16,8,36,18,4,9,3,7,2,4481,2706,2849,3274,2420,3574,3650,3390,3615,3881,2667,2889,3957,2889,2630,2990,3032,4063,3280,1186,873,628,919,726
mean,NaN,NaN,NaN,NaN,51.510889,-0.120217,NaN,NaN,76.337114,110.963663,203.426171,NaN,NaN,NaN,139.466209,79.027410,112.957172,24.831291,56.350769,70.993608,35.589744,0.187986,0.242596,0.757404,0.242806,0.757194,0.760134,0.239866,0.265700,0.734300,NaN,NaN,NaN,NaN,0.580130,0.190506,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [52]:
venues.head()

,space_url,venue_url,venue_name,space_name,latitude,longitude,address,nearest_tube_station,max_seated,max_standing,area_in_m2,catering_offered,external_catering_allowed,supervenue,Standing_max,Dining_max,Theatre_max,Boardroom_max,Classroom_max,Cabaret_max,U-Shaped_max,Approved caterers only,BYO alcohol allowed,BYO alcohol not allowed,External catering allowed,External catering not allowed,In-house catering,No in-house catering,Venue doesn’t provide alcohol,Venue provides alcohol,Alcohol licence until 23:00,Corkage fee for BYO alcohol,Buyout fee for external catering,Kitchen facilities available for guests,Complimentary water,Complimentary tea and coffee,Alcohol licence until 23:00 (extension available),Halal menu,Kosher menu,Extensive vegan menu,...,Alcohol licence until 22:30 (extension available),Alcohol licence until 3:00,Alcohol licence until 0:30 (extension available),Alcohol licence until 4:00,Alcohol licence until 23:30,Alcohol licence until 23:30 (extension available),Alcohol licence until 21:30,Alcohol licence until 4:00 (extension available),Alcohol licence until 2:30,Alcohol licence until 0:30,Alcohol licence until 22:00 (extension available),Alcohol licence until 5:00,Alcohol licence until 21:00 (extension available),Alcohol licence until 2:30 (extension available),Alcohol licence until 21:00,Alcohol licence until 1:30 (extension available),Wi-Fi,Projector,Flipchart,Natural light,Flatscreen TV,Whiteboard,Conference call facilities,Air conditioning,Storage space,Accommodation available,Parking available,Own music allowed,Bring your own DJ,PA system / music speakers available,Wheelchair accessible,Promoted / ticketed events,Loud music / events,Wedding licence,Temporary event notices (TENs) available,Paid parking facilities available nearby,Disabled access toilets,Ground level,Lift to all floors,Free parking is available on-site
0,https://www.tagvenue.com/rooms/london/19171/th...,https://www.tagvenue.com/venues/london/2034/th...,the golden hinde,entire ship,51.507014,-0.090456,"St Mary Overie Dock, Cathedral Street, London,...",London Bridge Station (350 yd),65,120,129,Venue doesn’t offer catering,External catering allowed,False,120.0,60.0,30.0,30.0,65.0,NaN,NaN,0,1,0,1,0,0,1,0,1,True,True,False,False,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,True,True,False,False,False,False,False,False,False,True,True,True,False,True,False,True,False,NaN,NaN,NaN,NaN,NaN
1,https://www.tagvenue.com/rooms/london/21285/go...,https://www.tagvenue.com/venues/london/9456/go...,goodenough college events & venue hire,large common room-wgh,51.524406,-0.117326,"Mecklenburgh Square, London House, London, WC1...",Russell Square Station (600 yd),120,120,135,Venue offers catering,External catering not allowed,False,120.0,NaN,120.0,35.0,35.0,60.0,35.0,0,1,0,0,1,1,0,0,1,NaN,False,NaN,NaN,0.0,0.0,True,True,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,True,True,False,False,True,False,True,True,False,True,False,True,True,True,True,False,False,NaN,NaN,NaN,NaN,NaN
2,https://www.tagvenue.com/rooms/london/1044/gra...,https://www.tagvenue.com/venues/london/410/gra...,gracepoint,whole building,51.540573,-0.096280,"161-169 Essex Road, Islington, London, N1 2SN",Highbury & Islington Station (900 yd),1160,0,1561,Venue offers catering,External catering not allowed,False,NaN,NaN,1160.0,NaN,NaN,NaN,NaN,1,0,1,0,1,1,0,1,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,False,True,False,False,False,True,False,False,False,True,False,True,True,True,True,True,True,NaN,NaN,NaN,NaN,NaN
3,https://www.tagvenue.com/rooms/london/4056/aqu...,https://www.tagvenue.com/venues/london/1635/aq...,aquilla health and fitness,studio,51.495765,-0.170124,"11 Thurloe Place, London, SW7 2RS",South Kensington Station (400 yd),0,12,35,Venue doesn’t offer catering,External catering not allowed,False,12.0,NaN,NaN,NaN,

In [ ]:
alreadsy checked bl;ah blah balh, not going to do it again...